<a href="https://colab.research.google.com/github/inyminymoe/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC)_%EA%B0%95%EC%9D%B8%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API 호출

In [11]:
import requests

## API 설정
api_key = '7657646572696e7931313561474d6b41'
year, month = 2024, 12
url = 'http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/'

r = requests.get(url, timeout=30)

## API 호출
response = requests.get(url)

data = response.json()

## 데이터 출력
print('URL:', url )
print('status_code:', r.status_code)
print('Content-Type:', r.headers.get('Content-Type'))
print('data:', data)

## 호출 성공/실패 출력
try:
    response.raise_for_status()
    print('API 호출 성공')

except requests.exceptions.HTTPError as e:
    print('API 호출 실패:', e)

js = response.json()

print("1. 바깥 폴더:", js.keys())
print("2. 안쪽 폴더:", js['energyUseDataStatInfo'].keys())
print("3. 데이터 타입:", type(js['energyUseDataStatInfo']['row']))
print("4. 데이터 개수:", len(js['energyUseDataStatInfo']['row']))

URL: http://openapi.seoul.go.kr:8088/7657646572696e7931313561474d6b41/json/energyUseDataStatInfo/1/5/2024/12/
status_code: 200
Content-Type: application/json;charset=UTF-8
data: {'energyUseDataStatInfo': {'list_total_count': 4110, 'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'}, 'row': [{'YEAR': '2024', 'MON': '12', 'HOSTCODE': '강남구', 'DONGCODE': '기타', 'MM_TYPE': '학교', 'TYPE_DIV': '단체', 'TYPE_DIV_CD': 'TB_GROUP', 'CNT': '1', 'EUS': '40385', 'EUS1': '38786', 'EUS2': '38383', 'ECO2_1_MATH_YEAR': '1599', 'ECO2_2_MATH_YEAR': '677.976', 'ECO2_1_MATH': '1800.5', 'ECO2_2_MATH': '763.412', 'ECO2_1': '1800.5', 'ECO2_2': '1800.5', 'E_TOE_YEAR': '0.36777', 'E_RATE_YEAR': '4.123', 'E_TOE_AVG': '0.414', 'E_RATE_AVG': '4.666', 'GUS': '3471', 'GUS1': '6841', 'GUS2': '5855', 'GCO2_1_MATH_YEAR': '-3370', 'GCO2_2_MATH_YEAR': '-7548.8', 'GCO2_1_MATH': '-2877', 'GCO2_2_MATH': '-6444.48', 'GCO2_1': '-2877', 'GCO2_2': '-5754', 'G_TOE_YEAR': '-3.515', 'G_RATE_YEAR': '-49.262', 'G_TOE_AVG': '-3.001', 

In [ ]:
## 설치 및 import
!pip install pandas requests

import requests, time
import panda as pd